# Gap Analysis for IDW Interpolation in Weekly and Monthly interval

* [1. Weekly Preprocess](#week)
* [2. Monthly Preprocess](#month)

In [62]:
import pandas as pd
import numpy as np
import arcpy
import rasterio
import pyproj
import glob
import warnings
from pyproj import Proj, transform
warnings.filterwarnings('ignore')

In [2]:
gis_path = r'F:\SEACAR_WQ_2024/GIS_Data/'
dfDis = pd.read_csv(gis_path + 'OEAT_Discrete_WQ-2024-May-06.csv', low_memory=False)

## 1. Weekly Preprocess <a class="anchor" id="week"></a>
### Aggregate the discrete data in same location as one row, and compute the period.

In [3]:
area_ab = ["GTM","EB","CH","BB","BBS"]
period_type = [" 52 week"," Month"]
def select_data_period(df,area,period):
    sheet_name = str(area) + str(period)
    df_period_table = pd.read_excel(gis_path + "All_Waterbodies_Season_Month_Week_Definitions.xlsx",sheet_name=sheet_name)
    df_select_area = df[df["WbodyAcronym"]==str(area)]
    df_period_table['Start Date'] = pd.to_datetime(df_period_table['Start Date'])
    df_period_table['End Date']   = pd.to_datetime(df_period_table['End Date'])
    sub_dfs = []

    for index, row in df_period_table.iterrows():
        start_date = row['Start Date']
        end_date   = row['End Date']
        sub_df = df_select_area[(df_select_area['SampleDate'] >= start_date) & (df_select_area['SampleDate'] < end_date)]
        sub_df['Period'] = row["Week"]
        sub_dfs.append(sub_df)
        
    df_period = pd.concat(sub_dfs,ignore_index=True)
    return df_period

In [4]:
dfDis['SampleDate'] = pd.to_datetime(dfDis['SampleDate'], format='%Y-%m-%d %H:%M:%S.%f')

In [5]:
sel_week_temp = []
for each in area_ab:
    df_week_temp = select_data_period(dfDis,str(each)," 52 week")
    df_week_temp_group = df_week_temp.groupby(['WaterBody','ParameterName','ParameterUnits',
                                          'Year','Season','Latitude_DD','Longitude_DD','WbodyAcronym',"Period"])["ResultValue"].agg("mean").reset_index()
    sel_week_temp.append(df_week_temp_group)
df_week_select_Mean = pd.concat(sel_week_temp,ignore_index=True)

C:\Users\xiang11\AppData\Local\Temp\ipykernel_14136\2361518724.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['Period'] = row["Week"]
C:\Users\xiang11\AppData\Local\Temp\ipykernel_14136\2361518724.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['Period'] = row["Week"]
C:\Users\xiang11\AppData\Local\Temp\ipykernel_14136\2361518724.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


C:\Users\xiang11\AppData\Local\Temp\ipykernel_14136\2361518724.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['Period'] = row["Week"]
C:\Users\xiang11\AppData\Local\Temp\ipykernel_14136\2361518724.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['Period'] = row["Week"]
C:\Users\xiang11\AppData\Local\Temp\ipykernel_14136\2361518724.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


C:\Users\xiang11\AppData\Local\Temp\ipykernel_14136\2361518724.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['Period'] = row["Week"]
C:\Users\xiang11\AppData\Local\Temp\ipykernel_14136\2361518724.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['Period'] = row["Week"]
C:\Users\xiang11\AppData\Local\Temp\ipykernel_14136\2361518724.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


In [11]:
param_shortnames = {
    'Salinity': 'Sal_ppt',
    'Total Nitrogen': 'TN_mgl',
    'Dissolved Oxygen': 'DO_mgl',
    'Turbidity':'Turb_ntu',
    'Secchi Depth':'Secc_m',
    'Water Temperature':'T_c'
}

In [13]:
df_week_select_Mean["ParaAcronym"] = df_week_select_Mean["ParameterName"].map(param_shortnames)

In [30]:
# Define the EPSG codes for source (EPSG:4326) and target (EPSG:3086) coordinate systems
source_epsg = 'EPSG:4326'
target_epsg = 'EPSG:3086'

# Create a PyProj Transformer for the conversion
transformer = pyproj.Transformer.from_crs(source_epsg, target_epsg, always_xy=True)

# Define a function to apply the transformation to each row of the DataFrame
def transform_coordinates(row):
    x, y = transformer.transform(row['Longitude_DD'], row['Latitude_DD'])
    return pd.Series({'x': x, 'y': y})

# Apply the transformation function to the DataFrame and create new columns for the converted coordinates
df_week_select_Mean[['x', 'y']] = df_week_select_Mean.apply(transform_coordinates, axis=1)

## Compute the total existing tiff files

In [26]:
raster_file_list = glob.glob(gis_path+"raster_output/idw_week/*.tif")
raster_name_list = []
for each in raster_file_list:
    name = each.split("\\")[-1]
    raster_name_list.append(name)

## Get the point value on the corresponding tiff file

In [49]:
value_list = []
for index,row in df_week_select_Mean.iterrows():
    wb     = row["WbodyAcronym"]
    para   = row["ParaAcronym"]
    year   = str(row["Year"])
    period = str(row["Period"])
    name   = str(f"{wb}_{para}_IDW_{year}_{period}.tif")
    x_cor  = row["x"]
    y_cor  = row["y"]
    path   = str(gis_path+"/raster_output/idw_week/"+name)
    if name in raster_name_list:
        with rasterio.open(path) as src:
            x,y = (x_cor,y_cor)
            row, col = src.index(x, y)
            value = src.read(1)[row, col]
            if value < -9999:
                value = ""
            else:
                value = value
            value_list.append(value)
    else:
        value = ""
        value_list.append(value)

In [51]:
df_week_select_Mean["ConValue"] = value_list

In [53]:
df_week_select_Mean.to_csv(gis_path + "week_difference.csv")

## 2. Monthly Preprocess <a class="anchor" id="month"></a>

In [55]:
area_ab = ["GTM","EB","CH","BB","BBS"]
period_type = [" 52 week"," Month"]
def select_data_period1(df,area,period):
    sheet_name = str(area) + str(period)
    df_period_table = pd.read_excel(gis_path + "All_Waterbodies_Season_Month_Week_Definitions.xlsx",sheet_name=sheet_name)
    df_select_area = df[df["WbodyAcronym"]==str(area)]
    df_period_table['Start Date'] = pd.to_datetime(df_period_table['Start Date'])
    df_period_table['End Date']   = pd.to_datetime(df_period_table['End Date'])
    sub_dfs = []

    for index, row in df_period_table.iterrows():
        start_date = row['Start Date']
        end_date   = row['End Date']
        sub_df = df_select_area[(df_select_area['SampleDate'] >= start_date) & (df_select_area['SampleDate'] < end_date)]
        sub_df['Period'] = row["Month"]
        sub_dfs.append(sub_df)
        
    df_period = pd.concat(sub_dfs,ignore_index=True)
    return df_period

In [56]:
sel_month_temp = []
for each in area_ab:
    df_month_temp = select_data_period1(dfDis,str(each)," Month")
    df_month_temp_group = df_month_temp.groupby(['WaterBody','ParameterName','ParameterUnits',
                                          'Year','Season','Latitude_DD','Longitude_DD','WbodyAcronym',"Period"])["ResultValue"].agg("mean").reset_index()
    sel_month_temp.append(df_month_temp_group)
df_month_select_Mean = pd.concat(sel_month_temp,ignore_index=True)

C:\Users\xiang11\AppData\Local\Temp\ipykernel_14136\3444084270.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['Period'] = row["Month"]
C:\Users\xiang11\AppData\Local\Temp\ipykernel_14136\3444084270.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['Period'] = row["Month"]
C:\Users\xiang11\AppData\Local\Temp\ipykernel_14136\3444084270.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

C:\Users\xiang11\AppData\Local\Temp\ipykernel_14136\3444084270.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['Period'] = row["Month"]
C:\Users\xiang11\AppData\Local\Temp\ipykernel_14136\3444084270.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['Period'] = row["Month"]
C:\Users\xiang11\AppData\Local\Temp\ipykernel_14136\3444084270.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [57]:
df_month_select_Mean["ParaAcronym"] = df_month_select_Mean["ParameterName"].map(param_shortnames)

In [58]:
# Define the EPSG codes for source (EPSG:4326) and target (EPSG:3086) coordinate systems
source_epsg = 'EPSG:4326'
target_epsg = 'EPSG:3086'

# Create a PyProj Transformer for the conversion
transformer = pyproj.Transformer.from_crs(source_epsg, target_epsg, always_xy=True)

# Define a function to apply the transformation to each row of the DataFrame
def transform_coordinates(row):
    x, y = transformer.transform(row['Longitude_DD'], row['Latitude_DD'])
    return pd.Series({'x': x, 'y': y})

# Apply the transformation function to the DataFrame and create new columns for the converted coordinates
df_month_select_Mean[['x', 'y']] = df_month_select_Mean.apply(transform_coordinates, axis=1)

In [59]:
raster_file_list_m = glob.glob(gis_path+"raster_output/idw_month/*.tif")
raster_name_list_m = []
for each in raster_file_list_m:
    name = each.split("\\")[-1]
    raster_name_list_m.append(name)

In [61]:
value_list_m = []
for index,row in df_month_select_Mean.iterrows():
    wb     = row["WbodyAcronym"]
    para   = row["ParaAcronym"]
    year   = str(row["Year"])
    period = str(row["Period"])
    name   = str(f"{wb}_{para}_IDW_{year}_{period}.tif")
    x_cor  = row["x"]
    y_cor  = row["y"]
    path   = str(gis_path+"/raster_output/idw_month/"+name)
    if name in raster_name_list:
        with rasterio.open(path) as src:
            x,y = (x_cor,y_cor)
            row, col = src.index(x, y)
            value = src.read(1)[row, col]
            if value < -9999:
                value = ""
            else:
                value = value
            value_list_m.append(value)
    else:
        value = ""
        value_list_m.append(value)

In [65]:
df_month_select_Mean["ConValue"] = value_list_m

In [66]:
df_month_select_Mean.to_csv(gis_path + "month_difference.csv")